
# Business Understanding

Corporacion Favorita is an Ecuador based company engaged in the organization, installation and administration of stores, markets and supermaektes. THe business areas in which the organiztion is involved are Commercial area, Real estate area and Industrial area. THe company aims to optimize its inventory management by accurately forecasting the product demand in various locations. 

---

By accurate demand forecasting, the company will be able to maintain optimal inventory levels, reduce wastage By aligning it supply with the customers demand, the company can enhance customer satisfaction, increase their sales revenue and reduce their carrying costs.
The key stakehoders are the Inventory managers, suppy chain managers, and store managers who will rely on the model of this project to make informed decisions related to purchasing, replenishment, and stock allocation.
The objectives of the project is to develop machine learning models to forecast the demand for products in various store locations of Corporation Favorita. The success of the project will depend on achieving the following outcomes:


* Accurate demand forecasting. The forecasts should consider varous factos like
the historical sales patters, seasonl variations, promotional activities,
location specific factos, and any other variable that might impact demand.


* Optimization of Inventory Levels. The forecast should gude inventory management decisions to maintain optimal stock levels.


* Timely and actionable insights. THe project should provide insights that will facilitate procative decision making


* Scalability and Adaptability: The model should be adaptable to accommodate changes in product assortments, market dynamics, and customer preferences over time.


# Data Description:

### train.csv

* The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.


* store_nbr identifies the store at which the products are sold.

* family identifies the type of product sold.

* sales gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).

* onpromotion gives the total number of items in a product family that were being promoted at a store at a given date.

### test.csv

* The test data, having the same features as the training data. You will predict the target sales for the dates in this file.

* The dates in the test data are for the 15 days after the last date in the training data.

### transaction.csv

* Contains date, store_nbr and transaction made on that specific date.
sample_submission.csv

* A sample submission file in the correct format.
stores.csv

* Store metadata, including city, state, type, and cluster.

* cluster is a grouping of similar stores.

### oil.csv

* Daily oil price which includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and its economical health is highly vulnerable to shocks in oil prices.)
holidays_events.csv

* Holidays and Events, with metadata
Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).
Additional Notes

* Wages in the public sector are paid every two weeks on the 15th and on the last day of the month. Supermarket sales could be affected by this.

* A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

# Questions
1. Is the train dataset complete (has all the required dates)?

2. Which dates have the lowest and highest sales for each year?

3. Did the earthquake impact sales?

4. Are certain groups of stores selling more products? (Cluster, city, state, type)

5. Are sales affected by promotions, oil prices and holidays?

6. What analysis can we get from the date and its extractable features?

7. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)

# Hypothesis 1
* H0 : Holiday lead to an increased sales
* H1 : Holidays do not lead to increased sales

# Hypothesis 2
* H0 : Earthquake impact sales
* H1 : Earthquake does not impact sales


In [95]:
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


## Importations

## Setting Up a Connection To a Remote Microsoft SQL Server

### Installation

In [96]:
#For creating a connection string with the remote database
!pip install  pyodbc

In [97]:
#Package for creating environment variables 
!pip install python-dotenv

### Importations

In [98]:
#packages for data handling
import pandas as pd
import numpy as np

import pyodbc
#For loading environment variables
from dotenv import dotenv_values


# For accessing the environment variables
import os
#TO filter warnings
import warnings
warnings.filterwarnings('ignore')

In [99]:
#Creating a new file .env
open('.env','w')

<_io.TextIOWrapper name='.env' mode='w' encoding='cp1252'>

In [100]:
#Writing the creditials in the .env file
f = open(".env", "a")
f.write("SERVER='dap-projects-database.database.windows.net'\n")
f.write("DATABASE='dapDB'\n")
f.write("USERNAME='dataAnalyst_LP3'\n")
f.write("PASSWORD='B0a@3kR$2z'\n")
f.close()

In [101]:
#opening the .env file
f = open(".env", "r")
print(f.read())

SERVER='dap-projects-database.database.windows.net'
DATABASE='dapDB'
USERNAME='dataAnalyst_LP3'
PASSWORD='B0a@3kR$2z'



In [102]:
# Loading environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')


# Getting the values for the credentials you set in the '.env' file
database = environment_variables.get("DATABASE")
server = environment_variables.get("SERVER")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")


connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [103]:
# Using the connect method of the pyodbc library to pass in the connection string to connect to the server

connection = pyodbc.connect(connection_string)

In [104]:
# Sql query to get the oil data

query = "Select * from dbo.oil"
oil = pd.read_sql(query, connection)

In [105]:
oil.head(5)

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.139999
2,2013-01-03,92.970001
3,2013-01-04,93.120003
4,2013-01-07,93.199997


In [106]:
oil.shape

(1218, 2)

In [107]:
oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [108]:
#Saving the oil dataset
oil.to_csv("oil.csv")

In [109]:
# Sql query to get stores data

query = "Select * from dbo.stores"
stores = pd.read_sql(query, connection)

In [110]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [111]:
stores.shape

(54, 5)

In [112]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


In [113]:
#Saving the stores data
stores.to_csv("stores.csv")

In [114]:
##Loading Holiday events data
query = "Select * From dbo.holidays_events"
holiday_events = pd.read_sql(query,connection)

In [115]:
holiday_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [116]:
holiday_events.shape

(350, 6)

In [117]:
holiday_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [118]:
#Saving holiday events dataset
holiday_events.to_csv("holiday_events")

In [119]:
# Loading test dataset
test = pd.read_csv('test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'test.csv'

In [ ]:
test.head()

In [ ]:
test.info()

In [ ]:
#Train dataset
train = pd.read_csv('train.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
#Transaction dataset
transaction = pd.read_csv('transactions.csv')

In [ ]:
transaction.head()

In [ ]:
transaction.info()

In [ ]:
vi